Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

*Note:* Each time we call the pipeline during grid search, each component is fit again. The vectorizer (tf-idf) is transforming our entire vocabulary during each cross-validation fold. That transformation adds significant run time to our grid search. There *might* be interactions between the vectorizer and our classifier, so we estimate their performance together in the code below. However, if your goal is to reduce run time. Train your vectorizer separately (ie out of the grid-searched pipeline). 

In [65]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from xgboost import XGBClassifier

from scipy.stats import randint
from scipy.stats import uniform

import pandas as pd
import numpy as np
import spacy

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [66]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [67]:
X_train = train.description
y_train = train.ratingCategory
X_test = test.description

### Define Pipeline Components

In [75]:
nlp = spacy.load('en_core_web_lg')

In [123]:
STOP_WORDS = nlp.Defaults.stop_words.union(['whiskey','whisky','drink','barrel','glass', 'bottle', 'll', 've'])

In [136]:
vect = TfidfVectorizer(stop_words=STOP_WORDS, ngram_range=(1,3))
rfc = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('rfc', rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [144]:
parameters = {
    'vect__max_df': uniform(.975, .999),
    'vect__min_df': uniform(.005, .015),
    'rfc__n_estimators': randint(200, 300),
    'rfc__max_features': uniform(0.05, 0.13),
    'rfc__max_depth': randint(34,36)
}

search = RandomizedSearchCV(pipe,parameters, cv=10, n_jobs=-1, verbose=1)
search.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 10.2min finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'vect__max_df': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000215E4D6D320>, 'vect__min_df': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000215EB8BB390>, 'rfc__n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000215EB8BB940>, 'rfc__max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000215EB8BB9E8>, 'rfc__max_depth': 

In [145]:
search.best_score_

0.7403963787619281

In [146]:
search.best_params_

{'rfc__max_depth': 35,
 'rfc__max_features': 0.16109491931884634,
 'rfc__n_estimators': 220,
 'vect__max_df': 1.7178875575024461,
 'vect__min_df': 0.00830125427092079}

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this compeition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [147]:
# Predictions on test sample
pred = search.predict(X_test)

In [148]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [149]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [150]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission10.csv', index=False)

## Challenge

You're trying to achieve 90% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [7]:
svd = TruncatedSVD(algorithm='randomized', n_iter=5)

In [14]:
parameters = {
    'lsi__svd__n_components': [10,25],
    'lsi__vect__min_df': (.05, 0.1),
    'lsi__vect__max_df': (0.75, 0.90),
    'clf__max_depth':(10,15,20)
}

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [15]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])

# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [16]:
grid_search = RandomizedSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.1min finished
C:\Users\JJ\Anaconda3\envs\U4-S1-NLP\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\JJ\Anaconda3\envs\U4-S1-NLP\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'lsi__svd__n_components': [10, 25], 'lsi__vect__min_df': (0.05, 0.1), 'lsi__vect__max_df': (0.75, 0.9), 'clf__max_depth': (10, 15, 20)},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=1)

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [54]:
# Predictions on test sample
pred = cross_val.predict(test['description'])

In [55]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [56]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [57]:
subNumber = 3

In [58]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [22]:
nlp = spacy.load("en_core_web_lg")

In [23]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [24]:
X = get_word_vectors(X_train)

len(X) == len(X_train)

True

In [25]:
len(X)

4087

In [26]:
# rfc.fit(X, y_train)

In [27]:
# rfc.score(X, y_train)

## Follow Along

In [35]:
xgb = XGBClassifier()

# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])

# Pipe
pipe = Pipeline([('lsi', lsi), ('xgb', xgb)])

In [49]:
params = {
    'lsi__vect__max_df' : (.85,.95),
    'lsi__vect__min_df' : (.025,.05),
    'lsi__vect__max_features': (500,1000),
    'xgb__learning_rate': (.1,.2,.4),
    'xgb__max_depth':(3,6,9),
}

In [51]:
cross_val = GridSearchCV(pipe, params, cv=5, n_jobs=-1, verbose=1)

In [52]:
cross_val.fit(X_train,y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  7.7min finished
C:\Users\JJ\Anaconda3\envs\U4-S1-NLP\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm=...
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'lsi__vect__max_df': (0.85, 0.95), 'lsi__vect__min_df': (0.025, 0.05), 'lsi__vect__max_features': (500, 1000), 'xgb__learning_rate': (0.1, 0.2, 0.4), 'xgb__max_depth': (3, 6, 9)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [53]:
cross_val.best_score_

0.704183998042574

## New Attempt

In [76]:
def get_tokens(docs):
    tokenized = []
    for doc in docs:
        nlp_doc = nlp(doc)
        tokens = ''
        for token in nlp_doc:
            if (token.is_stop is not True) and (token.is_punct is not True):
                tokens = tokens + ' ' + token.text.lower()
        tokenized.append(tokens)
    return tokenized

In [78]:
nlp(X_train[0])[21]

pulled

In [79]:
X_train_token = get_tokens(X_train)

In [80]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.01, max_df=0.7, max_features=500)

In [81]:
X_train_vect = vect.fit_transform(X_train_token)

In [82]:
from sklearn.model_selection import train_test_split

X_train_vect, X_val_vect = train_test_split(X_train_vect, shuffle=False)

In [83]:
y_train, y_val = train_test_split(y_train, shuffle=False)

In [84]:
eval_set = [(X_train_vect, y_train),
            (X_val_vect, y_val)]

model = XGBClassifier(
    n_estimators=1000,
    max_depth=5,
    learning_rate=0.1,
    n_jobs=-1
)

model.fit(X_train_vect, y_train, eval_set=eval_set, eval_metric='merror', early_stopping_rounds=50)

[0]	validation_0-merror:0.250897	validation_1-merror:0.295499
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 50 rounds.
[1]	validation_0-merror:0.248613	validation_1-merror:0.286693
[2]	validation_0-merror:0.252529	validation_1-merror:0.276908
[3]	validation_0-merror:0.249918	validation_1-merror:0.273973
[4]	validation_0-merror:0.246982	validation_1-merror:0.273973
[5]	validation_0-merror:0.244698	validation_1-merror:0.272994
[6]	validation_0-merror:0.243719	validation_1-merror:0.271037
[7]	validation_0-merror:0.241762	validation_1-merror:0.279843
[8]	validation_0-merror:0.241436	validation_1-merror:0.271037
[9]	validation_0-merror:0.242414	validation_1-merror:0.272994
[10]	validation_0-merror:0.241762	validation_1-merror:0.272016
[11]	validation_0-merror:0.241109	validation_1-merror:0.271037
[12]	validation_0-merror:0.241109	validation_1-merror:0.271037
[13]	validation_0-merror:0.2

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=1000, n_jobs=-1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [85]:
X_train_vect = vect.fit_transform(X_train_token)

In [87]:
y_train = train['ratingCategory']

In [88]:
model = XGBClassifier(
    n_estimators=1000,
    max_depth=5,
    learning_rate=0.1,
    n_jobs=-1
)

model.fit(X_train_vect, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=1000, n_jobs=-1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [89]:
X_test_token = get_tokens(X_test)

In [90]:
X_test_vect = vect.transform(X_test_token)

In [91]:
pred = model.predict(X_test_vect)

In [92]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')
submission.to_csv('./data/submission5.csv', index=False)

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle comeptition
* Find another text classification task to work on